In [1]:
import torch
from torch.utils.data import DataLoader
import lava.lib.dl.slayer as slayer
from nmnist import NMNIST, SNN, augment

         Only Prophesee DVS demo will not run properly.
         Please install it from https://github.com/prophesee-ai/prophesee-automotive-dataset-toolbox


In [3]:
trained_dir = '.'
nmnist = NMNIST(trained_dir, 3, 300, transform=augment, download=True)
nmnist_test = NMNIST(trained_dir, 3, 300)

device = torch.device('cpu')

net = SNN().to(device)

optimizer = torch.optim.Adam(net.parameters(), lr=0.001)

nmnist_loader = DataLoader(
    dataset=nmnist, batch_size=32, shuffle=True
)

nmnist_test_loader = DataLoader(
    dataset=nmnist, batch_size=32, shuffle=True
)

error = slayer.loss.SpikeRate(
    true_rate=0.2, false_rate=0.03, reduction='sum'
).to(device)

stats = slayer.utils.LearningStats()
assistant = slayer.utils.Assistant(
    net, error, optimizer, stats,
    classifier=slayer.classifier.Rate.predict, count_log=True
)

epochs = 1

for epoch in range(epochs):
    for i, (input, label) in enumerate(nmnist_loader):
        output, count = assistant.train(input, label)
        header = [
            'TRAIN\nEvent rate : ' +
            ', '.join([f'{c.item():.4f}' for c in count.flatten()]),
            'Output : ' + str(output[0].sum(dim=1) / output[0].sum()),
            'Label : ' + str(label[0])
        ]
        stats.print(epoch, iter=i, header=header, dataloader=nmnist_loader)

    for i, (input, label) in enumerate(nmnist_test_loader):
        output, count = assistant.test(input, label)
        header = [
            'TEST\nEvent rate : ' +
            ', '.join([f'{c.item():.4f}' for c in count.flatten()]),
            'Output : ' + str(output[0].sum(dim=1) / output[0].sum()),
            'Label : ' + str(label[0])
        ]
        stats.print(epoch, iter=i, header=header, dataloader=nmnist_loader)

    if stats.testing.best_accuracy:
        torch.save(net.state_dict(), trained_dir + '/network.pt')

    stats.update()

net.export_hdf5('network.net')

Attempting download...



     0K .......... .......... .......... .......... ..........  0%  393K 41m53s
    50K .......... .......... .......... .......... ..........  0%  656K 33m29s
   100K .......... .......... .......... .......... ..........  0% 1.47M 25m58s
   150K .......... .......... .......... .......... ..........  0% 1.34M 22m28s
   200K .......... .......... .......... .......... ..........  0% 2.23M 19m25s
   250K .......... .......... .......... .......... ..........  0% 2.51M 17m15s
   300K .......... .......... .......... .......... ..........  0% 2.89M 15m35s
   350K .......... .......... .......... .......... ..........  0% 1.76M 14m46s
   400K .......... .......... .......... .......... ..........  0% 2.76M 13m47s
   450K .......... .......... .......... .......... ..........  0% 6.15M 12m40s
   500K .......... .......... .......... .......... ..........  0% 2.21M 12m10s
   550K .......... .......... .......... .......... ..........  0% 6.25M 11m22s
   600K .......... .......... .........

Extracting files ...
Download complete.

TRAIN
Event rate : 0.0062, 0.0033, 0.0008
Output : tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       grad_fn=<DivBackward0>)
Label : tensor(9)
Epoch    0:     0/60000 (0%)        
Train loss =     1.52510                          accuracy = 0.06250 

TRAIN
Event rate : 0.0145, 0.0153, 0.1273
Output : tensor([0.1243, 0.1017, 0.0960, 0.1073, 0.0339, 0.1243, 0.1243, 0.1017, 0.1073,
        0.0791], grad_fn=<DivBackward0>)
Label : tensor(2)
Epoch    0:    32/60000 (0%)        
Train loss =     2.85430                          accuracy = 0.07812 

TRAIN
Event rate : 0.0082, 0.0061, 0.0325
Output : tensor([nan, nan, nan, nan, nan, nan, nan, nan, nan, nan],
       grad_fn=<DivBackward0>)
Label : tensor(1)
Epoch    0:    64/60000 (0%)        
Train loss =     2.30520                          accuracy = 0.08333 

TRAIN
Event rate : 0.0069, 0.0048, 0.0117
Output : tensor([0.0556, 0.0000, 0.0556, 0.0000, 0.4444, 0.0000, 0.1111, 0.0000, 0.27

KeyboardInterrupt: 

In [3]:
nmnist.example_of_each_class(assistant)